# Задача о расписании

### Метод перебором
Цель: найти расписание, при котором ожидание автобуса в час пик <= 5 мин.

In [ ]:
from datetime import datetime, timedelta, time
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QTableWidget, QTableWidgetItem, QVBoxLayout, QWidget, QPushButton, QComboBox, QLabel, QTabWidget, QLineEdit, QMessageBox
from PyQt5 import QtGui


class DataInputWindow(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Ввод данных")
        self.setGeometry(100, 100, 400, 300)

        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        # Поля ввода для автобусов
        self.drivers_count = QLineEdit()
        self.drivers_count.setPlaceholderText("Мин. водителей (>0)")
        
        self.bus_count = QLineEdit()
        self.bus_count.setPlaceholderText("Мин. автобусов (>0)")
        
        self.route_time = QLineEdit()
        self.route_time.setPlaceholderText("Время автобуса в пути (в часах)")

        self.peak_interval = QLineEdit()
        self.peak_interval.setPlaceholderText("Макс. среднее время между автобусами в час пик (в минутах)")

        layout.addWidget(self.drivers_count)
        layout.addWidget(self.bus_count)
        layout.addWidget(self.route_time)
        layout.addWidget(self.peak_interval)

        # Кнопка сохранения
        self.save_button = QPushButton("Показать расписание")
        self.save_button.clicked.connect(self.save_data)

        layout.addWidget(self.save_button)
        self.setLayout(layout)

    def save_data(self):
        if int(self.drivers_count.text()) <= 0 or int(self.bus_count.text()) <= 0:
            QMessageBox.warning(self, "Ошибка", "Заполните все поля правильно")
        # Получаем введенные данные
        input_data = [
            int(self.drivers_count.text()),
            int(self.bus_count.text()),
            int(self.route_time.text()),
            int(self.peak_interval.text()),
        ]

        if all(input_data):
            # Если все поля заполнены, передаем данные в главное окно
            data = get_schedule(input_data[0], input_data[1], timedelta(hours=input_data[2]), timedelta(minutes=input_data[3]) )

            self.main_window = BusScheduleApp(data)
            self.main_window.show()
            self.close()  # Закрываем текущее окно

            self.close()  # Закрываем текущее окно

        else:
            QMessageBox.warning(self, "Ошибка", "Пожалуйста, заполните все поля.")

class BusScheduleApp(QMainWindow):
    def __init__(self, data):
        super().__init__()
        
        self.setWindowTitle("Расписание (Решение в лоб)")
        self.setWindowIcon(QtGui.QIcon('logo.png'))
        self.setGeometry(100, 100, 600, 400)

        self.initUI(data)

    def initUI(self, data):
        # Создание виджета и компоновщика
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        # Создание вкладок
        self.tabs = QTabWidget()
        self.layout.addWidget(self.tabs)

        # Добавление вкладки с расписанием автобусов
        self.bus_schedule_tab = QWidget()
        self.tabs.addTab(self.bus_schedule_tab, "Расписание автобусов")
        self.bus_schedule_layout = QVBoxLayout(self.bus_schedule_tab)

        # Создание выпадающего списка для выбора дня недели
        self.day_selector = QComboBox()
        days_of_week = ["Все", "Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
        self.day_selector.addItems(days_of_week)
        self.day_selector.currentIndexChanged.connect(self.update_bus_table)
        self.day_selector.currentIndexChanged.connect(self.update_driver_table)

        # Добавление выпадающего списка в компоновщик
        self.layout.addWidget(self.day_selector)

        # Добавление заголовка для расписания автобусов
        self.bus_title_label = QLabel("Расписание движения автобусов")
        self.bus_title_label.setStyleSheet("font-size: 20px; font-weight: bold;")
        self.bus_schedule_layout.addWidget(self.bus_title_label)

        # Добавление заголовка
        self.info = QLabel("")
        self.info.setStyleSheet("font-size: 10px; font-weight: bold;")
        self.layout.addWidget(self.info)

        # Создание таблицы для расписания автобусов
        self.bus_table = QTableWidget()
        self.bus_table.setColumnCount(4)
        self.bus_table.setHorizontalHeaderLabels(["День недели", "Номер автобуса", "Время прибытия", "Средний интервал в час пик"])

        # заполнение таблицы данными
        self.populate_bus_table(data)

        # Добавление таблицы в компоновщик
        self.bus_schedule_layout.addWidget(self.bus_table)

        # Добавление вкладки с расписанием водителей
        self.driver_schedule_tab = QWidget()
        self.tabs.addTab(self.driver_schedule_tab, "Расписание водителей")
        self.driver_schedule_layout = QVBoxLayout(self.driver_schedule_tab)

        # Добавление заголовка для расписания водителей
        self.driver_title_label = QLabel("Расписание водителей")
        self.driver_title_label.setStyleSheet("font-size: 20px; font-weight: bold;")
        self.driver_schedule_layout.addWidget(self.driver_title_label)

        # Создание таблицы для расписания водителей
        self.driver_table = QTableWidget()
        self.driver_table.setColumnCount(5)
        self.driver_table.setHorizontalHeaderLabels(["День недели", "ID водителя", "Время", "Состояние", "Автобус"])

        # Заполнение таблицы данными
        self.populate_driver_table(data)

        # Добавление таблиц в компоновщик
        self.layout.addWidget(self.driver_table)
        self.driver_schedule_layout.addWidget(self.driver_table)

    def populate_driver_table(self, data=[]):
        if len(data) != 0:
            days_names = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
            self.driver_schedule_data = []
            drivers_list = []
            for i in range(7):
                for j in range(len(data["buses_schedule"][i])):
                        driver = data["buses_schedule"][i][j][0]
                        if driver not in drivers_list:
                            for z in range(len(driver.schedule)):
                                driver_time = str(driver.schedule[z][0])
                                if "1 day" in driver_time:
                                    driver_time = driver_time.replace("1 day,", "")
                                self.driver_schedule_data.append((days_names[i], str(driver.id), driver_time, driver.schedule[z][1], str(driver.bus))) 
                            drivers_list.append(driver)
                    
            self.info.setText(f"Всего автобусов: {data["buses_count"][0]}\nВсего водителей: {data["drivers_count"][0]}")
            self.update_driver_table()

    def populate_bus_table(self, data=[]):
        if len(data) != 0:
            days_names = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
            self.bus_schedule_data = []
            for i in range(7):
                for j in range(len(data["buses_schedule"][i])):
                    bus = data["buses_schedule"][i][j]
                    bus_driver = bus[0]
                    bus_time = str(bus[1])
                    if "1 day" in bus_time:
                        bus_time = bus_time.replace("1 day,", "")
                    self.bus_schedule_data.append((days_names[i], str(bus_driver.bus), bus_time, data["peak_intervals"][i])) 
                        
            self.info.setText(f"Всего автобусов: {data["buses_count"][0]}\nВсего водителей: {data["drivers_count"][0]}")

            self.update_bus_table()

    def update_bus_table(self):
        selected_day = self.day_selector.currentText()
        
        # Очищаем таблицу перед заполнением
        self.bus_table.setRowCount(0)

        for day, bus_number, time, interval in self.bus_schedule_data:
            if selected_day == "Все" or day == selected_day:
                
                row_position = self.bus_table.rowCount()
                self.bus_table.insertRow(row_position)
                self.bus_table.setItem(row_position, 0, QTableWidgetItem(day))
                self.bus_table.setItem(row_position, 1, QTableWidgetItem(str(bus_number)))
                self.bus_table.setItem(row_position, 2, QTableWidgetItem(str(time)))
                self.bus_table.setItem(row_position, 3, QTableWidgetItem(str(interval).split(".")[0].split(":", 1)[1]))
    
    def update_driver_table(self):
        selected_day = self.day_selector.currentText()
        
        # Очищаем таблицу перед заполнением
        self.driver_table.setRowCount(0)

        for day, id, time, state, bus in self.driver_schedule_data:
            if selected_day == "Все" or day == selected_day:
                row_position = self.driver_table.rowCount()
                self.driver_table.insertRow(row_position)
                self.driver_table.setItem(row_position, 0, QTableWidgetItem(day))
                self.driver_table.setItem(row_position, 1, QTableWidgetItem(id))
                self.driver_table.setItem(row_position, 2, QTableWidgetItem(str(time).split(".")[0]))
                self.driver_table.setItem(row_position, 3, QTableWidgetItem(state))
                self.driver_table.setItem(row_position, 4, QTableWidgetItem(bus))

class Driver_A:
    count = 0
    all_drivers = []

    def __init__(self, bus=None):
        self.work_days = []
        self.bus = bus
        self.work_time = timedelta(hours=8.0)
        self.lunch_time = timedelta(hours=1.0)
        self.min_time_bf_dinner = timedelta(hours=4.0)
        self.type = "A"
        self.state = 0
        self.schedule = []
        self.hunger = True
        self.id = Driver_A.count
        Driver_A.count += 1
        Driver_A.all_drivers.append(self)

    def __repr__(self):
        return str(self.id) + " " + str(self.type) + " " + str(self.bus)

    def start(self, time, bus,all_buses_exit_schedule ):
        self.work_start_time = time
        self.start_time = time
        self.state = 1
        self.bus = bus
        self.bus.driver = self
        
        self.schedule.append([time, "Выход"])
        self.bus.schedule.append([time, "Выход с автопарка"])
        all_buses_exit_schedule.append([self, time])

    
    def check(self, current_time, free_buses, busy_buses, all_buses_exit_schedule):
        if self.state != 0:
    
            if (current_time - self.work_start_time) >= (self.work_time+self.lunch_time):
                self.state = 0
                self.schedule.append([current_time, "Уход"])
                self.bus.driver = None
                free_buses.append(self.bus)
                busy_buses.remove(self.bus)
                self.bus.schedule.append([current_time, "Возвращение в автопарк"])
               
            if self.state == 1:
                if current_time - self.start_time >= route_time*2:
                    self.bus.schedule.append([current_time, "Возвращение в автопарк"])
                    if self.need_rest(current_time):
                        self.state = 2
                        self.start_time = current_time
                        self.schedule.append([current_time, "Перерыв"])
                       
                    else:
                        self.bus.schedule.append([current_time, "Выход с автопарка"])
                       
                        all_buses_exit_schedule.append([self, current_time])
                        self.start_time = current_time
            elif self.state == 2:
                if current_time - self.start_time >= self.lunch_time:
                    self.hunger = False
                    self.state = 1
                    self.start_time = current_time
                    self.schedule.append([current_time, "Выход"])
                    self.bus.schedule.append([current_time, "Выход с автопарка"])
                    all_buses_exit_schedule.append([self, current_time])
                   
            

    def need_rest(self, current_time):
        if self.work_start_time:
            if (self.hunger == True) and (current_time - self.work_start_time >= self.min_time_bf_dinner):
                return True
        return False  

class Driver_B:
    count = 0
    all_drivers = []

    def __init__(self, bus=None):
        self.work_days = []
        self.bus = bus
        self.work_time = timedelta(hours=12.0)
        self.lunch_time = timedelta(minutes=15.0)
        self.lunch_time_interval = timedelta(hours=2.0) # минимум
        self.type = "B"
        self.state = 0
        self.schedule = []
        self.hunger = True
        Driver_B.count += 1
        self.id = Driver_B.count
        Driver_B.all_drivers.append(self)
        # Обед во время одного из перерывов
        # 1 сутки работает, 2 отдыхает
    
    def __repr__(self):
        return str(self.id) + " " + str(self.type) + " " + str(self.bus)

    def start(self, time, bus, all_buses_exit_schedule):
        self.work_start_time = time
        self.start_time = time
        self.state = 1
        self.bus = bus
        self.bus.driver = self
        
        
        self.schedule.append([time, "Выход"])
        self.bus.schedule.append([time, "Старт с автопарка"])
        all_buses_exit_schedule.append([self, time])



    def check(self, current_time, free_buses, busy_buses, all_buses_exit_schedule):
        if self.state != 0:
            if (current_time - self.work_start_time) >= (self.work_time+5*self.lunch_time):
                self.state = 0
                self.schedule.append([current_time, "Уход"])
                self.bus.driver = None
                free_buses.append(self.bus)
                busy_buses.remove(self.bus)
                self.bus.schedule.append([current_time, "Возвращение в автопарк"])
               
            if self.state == 1:
                if current_time - self.start_time >= route_time*2:
                    self.bus.schedule.append([current_time, "Возвращение в автопарк"])
                    if self.need_rest(current_time):
                        self.state = 2
                        self.start_time = current_time
                        self.schedule.append([current_time, "Перерыв"])
                        
                    else:
                        self.bus.schedule.append([current_time, "Выход с автопарка"])
                        all_buses_exit_schedule.append([self, current_time])
                        self.start_time = current_time
                        
            elif self.state == 2:
                if current_time - self.start_time >= self.lunch_time:
                    self.hunger = False
                    self.state = 1
                    self.start_time = current_time
                    self.bus.schedule.append([current_time, "Выход с автопарка"])
                    all_buses_exit_schedule.append([self, current_time])
                 

    def need_rest(self, current_time):
        if self.start_time:
            if (self.hunger == True) and (current_time - self.start_time >= self.lunch_time_interval):
                return True
        return False

class Bus:
    count = 0
    all_buses = []

    def __init__(self):
        Bus.count += 1
        self.schedule = []
        self.id = Bus.count
        self.driver = None
        Bus.all_buses.append(self)
        
    def __repr__(self):
        return str(self.id)

def get_drivers_choice(drivers_count):
    drivers = []
    day_index_list_A = [1, 2, 3]
    day_index_list_B = [1, 2, 3]
    day_index_A = 0
    day_index_B = 0
    drivers_B = []
    for drivers_B_Count in range(0, drivers_count+1): # выбирается количество водителей типа B
        if drivers_B_Count != 0:
            driver_B = Driver_B()
            driver_B.work_days = [day_index_list_B[day_index_B]+i for i in range(0, 4, 3)]
            if day_index_B == 2:
                day_index_B = 0
            else:
                day_index_B += 1  
            drivers_B.append(driver_B)

        drivers_A = []
        day_index_A = 0
        for i in range(0, drivers_count-drivers_B_Count): # выбирается количество водителей типа A
            driver_A = Driver_A()
            driver_A.work_days = [day_index_list_A[day_index_A]+j for j in range(5)]
            
            if day_index_A == 2:
                day_index_A = 0
            else:
                day_index_A += 1
            drivers_A.append(driver_A)
        append_list_A = drivers_A.copy()
        append_list_B = drivers_B.copy()
        drivers.append([append_list_A, append_list_B])
    return drivers

def get_driver_by_type(drivers_list: list, type: str):
    if len(drivers_list) == 0:
        return False
    for driver in drivers_list:
        if driver.type == type:
            drivers_list.remove(driver)
            return driver

# переменные
buses_count = 1
drivers_count = 1
route_time = timedelta(hours=1.0)
peak_max_interval = timedelta(minutes=5.0)

# часы работы автопарка
start_time = timedelta(hours=6.0)
finish_time = timedelta(days=1, hours=3.0)

# часы пик ПН-ПТ
peak_hour_starts = [timedelta(hours=7.0), timedelta(hours=17.0)]
peak_hour_finishes = [timedelta(hours=9.0), timedelta(hours=19.0)]

A_drivers_all = []
B_drivers_all = []
buses_schedule_all = []
drivers_count_all = []
buses_count_all = []
peak_hour_interval_all = []

def get_schedule(buses_count=1, drivers_count=1, route_time=timedelta(hours=1.0), peak_max_interval=timedelta(minutes=5.0)):
    route_time = route_time
    for day_index in range(1, 8):
        buses_schedule = []
        A_drivers = []
        B_drivers = []
        peak_interval = timedelta(hours=1.0)

        while peak_interval > peak_max_interval:
            all_buses_exit_schedule = []
            free_drivers = []
            free_buses = []
            busy_buses = []
            planned_time = []
            drivers_A_count = 0
            drivers_B_count = 0
            current_time = start_time

            drivers = get_drivers_choice(drivers_count)
            
            # сбор водителей этого дня в массив и подсчет количества водителей разных типов
            for drivers_list in drivers[drivers_count//2]:
                for driver in drivers_list:
                    if day_index in driver.work_days:
                        free_drivers.append(driver)
                        if driver.type == "A":
                            drivers_A_count += 1
                        if driver.type == "B":
                            drivers_B_count += 1

            # сбор автобусов в массив
            for i in range(buses_count):
                free_buses.append(Bus())

        
            # счетчик времени прибавляет по 5 мин. пока не дойдет до закрытия автопарка
            while current_time <= finish_time:
                # в начале проверяем, если много водителей, разделяем их равномерно по часам пик
                # водители типа А
                if current_time == start_time and len(free_buses) != 0:
                    if drivers_A_count > 3:
                        offset = timedelta(minutes=5.0)
                        if drivers_A_count == 4:
                            planned_time.append([start_time+offset, "A"])
                        for i in range((drivers_A_count-3)//2):
                            planned_time.append([start_time+offset, "A"])
                            offset += timedelta(minutes=5.0)
                        offset = timedelta(minutes=5.0)
                        for i in range(len(free_drivers)-((drivers_A_count-3)//2)):
                            planned_time.append([peak_hour_starts[1]+offset, "A"])
                            offset += timedelta(minutes=5.0)
                # водители типа B
                    if drivers_B_count > 1:
                        offset = timedelta(minutes=5)
                        for i in range((drivers_B_count-1)):
                            planned_time.append([start_time+timedelta(hours=9.0)+offset, "B"])
                            offset += timedelta(minutes=10.0)
                
                if len(planned_time) != 0:
                    for p_time in planned_time:
                        if current_time == p_time[0] and len(free_buses) != 0 and len(free_drivers) !=0:
                            driver = get_driver_by_type(free_drivers, p_time[1])
                            if driver:
                                bus =  free_buses[0]
                                driver.start(current_time, bus, all_buses_exit_schedule)
                                busy_buses.append(bus)
                                del free_buses[0]

                if ((current_time == start_time) or (current_time == start_time+timedelta(hours=9.0)) or\
                    (current_time == start_time+timedelta(hours=11.0))) and len(free_buses) != 0:
                    
                    get_drivers_list = [get_driver_by_type(free_drivers, "A"), get_driver_by_type(free_drivers, "B")]
                    if current_time != start_time:
                        get_drivers_list = get_drivers_list[:1]
                        
                    for driver in get_drivers_list:
                        if driver:
                            bus =  free_buses[0]
                            driver.start(current_time, bus, all_buses_exit_schedule)
                            busy_buses.append(bus)
                            del free_buses[0]

                # проверка состояния активных автобусов 
                if len(busy_buses) != 0:
                    for bus in busy_buses:
                        bus.driver.check(current_time, free_buses, busy_buses, all_buses_exit_schedule)
                
                # шаг по времени на 5 мин.
                current_time += timedelta(minutes=5.0)

            summ = timedelta(hours=0.0)
            day_interval = timedelta(hours=0.0)
            for i in range(1, len(all_buses_exit_schedule)):
                summ += (all_buses_exit_schedule[i][1] - all_buses_exit_schedule[i-1][1])
            if len(all_buses_exit_schedule) != 0:
                day_interval = summ/len(all_buses_exit_schedule)
              

            peak_summ = timedelta(hours=0.0)
            for i in range(1, len(all_buses_exit_schedule)):
                if timedelta(hours=7.0) < all_buses_exit_schedule[i][1] < timedelta(hours=9.0) or \
                    timedelta(hours=17.0) < all_buses_exit_schedule[i][1] < timedelta(hours=19.0):
                    peak_summ += (all_buses_exit_schedule[i][1] - all_buses_exit_schedule[i-1][1])
            if len(all_buses_exit_schedule) != 0:
                peak_interval = peak_summ/len(all_buses_exit_schedule)
              

            buses_schedule = all_buses_exit_schedule
            
            for driver in Driver_A.all_drivers:
                A_drivers.append(driver)
            for driver in Driver_B.all_drivers:
                B_drivers.append(driver)
            
            if peak_interval > peak_max_interval:
                drivers_count += 2
                buses_count += 1

        A_drivers_all.append(A_drivers)
        B_drivers_all.append(B_drivers)
        buses_schedule_all.append(buses_schedule)

        drivers_count_all.append(drivers_count)
        buses_count_all.append(buses_count)
        peak_hour_interval_all.append(peak_interval)

    data = {
        "buses_schedule": buses_schedule_all,
        "peak_intervals": peak_hour_interval_all,
        "drivers_count": drivers_count_all,
        "buses_count": buses_count_all,
    }
    return data

app = QApplication(sys.argv)
input_window = DataInputWindow()
input_window.show()
sys.exit(app.exec_())

### Генетический алгоритм
Цель: найти расписание, где наименьшее ожидание автобуса в час пик

In [1]:
from datetime import datetime, timedelta, time
import random
from PyQt5.QtWidgets import QApplication, QMainWindow, QTableWidget, QTableWidgetItem, QVBoxLayout, QWidget, QPushButton, QComboBox, QLabel, QTabWidget, QLineEdit, QMessageBox
from PyQt5 import QtGui
import sys


class DataInputWindow(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Ввод данных")
        self.setGeometry(100, 100, 400, 300)

        self.initUI()

    def initUI(self):
        layout = QVBoxLayout()

        # Поля ввода для автобусов
        self.drivers_count = QLineEdit()
        self.drivers_count.setPlaceholderText("Мин. водителей (>0)")
        
        self.bus_count = QLineEdit()
        self.bus_count.setPlaceholderText("Мин. автобусов (>0)")
        
        self.route_time = QLineEdit()
        self.route_time.setPlaceholderText("Время автобуса в пути (в часах)")


        layout.addWidget(self.drivers_count)
        layout.addWidget(self.bus_count)
        layout.addWidget(self.route_time)
  

        # Кнопка сохранения
        self.save_button = QPushButton("Показать расписание")
        self.save_button.clicked.connect(self.save_data)

        layout.addWidget(self.save_button)
        self.setLayout(layout)

    def save_data(self):
        if int(self.drivers_count.text()) <= 0 or int(self.bus_count.text()) <= 0:
            QMessageBox.warning(self, "Ошибка", "Заполните все поля правильно")
        # Получаем введенные данные
        input_data = [
            int(self.drivers_count.text()),
            int(self.bus_count.text()),
            int(self.route_time.text()),
         
        ]

        if all(input_data):
            # Если все поля заполнены, передаем данные в главное окно
            data = get_schedule(input_data[0], input_data[1], timedelta(hours=input_data[2]))

            self.main_window = BusScheduleApp(data)
            self.main_window.show()                 # ВАЖНО
            self.close()  # Закрываем текущее окно
            self.close()  # Закрываем текущее окно  # ВАЖНО

        else:
            QMessageBox.warning(self, "Ошибка", "Пожалуйста, заполните все поля.")


class BusScheduleApp(QMainWindow):
    def __init__(self, data):
        super().__init__()
        
        self.setWindowTitle("Расписание (Генетический алгоритм)")
        self.setWindowIcon(QtGui.QIcon('logo.png'))
        self.setGeometry(100, 100, 600, 400)

        self.initUI(data)

    def initUI(self, data):
        # Создание виджета и компоновщика
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.layout = QVBoxLayout(self.central_widget)

        # Добавление заголовка
        self.info = QLabel("")
        self.info.setStyleSheet("font-size: 10px; font-weight: bold;")
        self.layout.addWidget(self.info)

        # Создание таблицы для расписания автобусов
        self.bus_table = QTableWidget()
        self.bus_table.setColumnCount(4)
        self.bus_table.setHorizontalHeaderLabels(["День недели", "Номер автобуса", "Время прибытия", "Средний интервал в час пик"])

        # заполнение таблицы данными
        self.populate_bus_table(data)

        # Добавление таблицы в компоновщик
        self.layout.addWidget(self.bus_table)

    def populate_bus_table(self, data=[]):
        if len(data) != 0:
            data, peak_interval = data[0], str(data[1])
            days_names = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
            self.bus_schedule_data = []
            for i in range(len(data)):
                for j in range(len(data[i])):
                    bus = data[i][j][0].bus
                    bus_time = str(data[i][j][1])
                    if "1 day" in bus_time:
                        bus_time = bus_time.replace("1 day,", "")
                    self.bus_schedule_data.append((days_names[0], str(bus), bus_time, peak_interval)) 
                        
            self.update_bus_table()

    def update_bus_table(self):
  
        
        # Очищаем таблицу перед заполнением
        self.bus_table.setRowCount(0)

        for day, bus_number, time, interval in self.bus_schedule_data:
 
                
            row_position = self.bus_table.rowCount()
            self.bus_table.insertRow(row_position)
            self.bus_table.setItem(row_position, 0, QTableWidgetItem(day))
            self.bus_table.setItem(row_position, 1, QTableWidgetItem(str(bus_number)))
            self.bus_table.setItem(row_position, 2, QTableWidgetItem(str(time)))
            self.bus_table.setItem(row_position, 3, QTableWidgetItem(str(interval).split(".")[0].split(":", 1)[1]))


class Driver_A:
    count = 0
    all_drivers = []

    def __init__(self, bus=None):
        self.work_days = []
        self.bus = bus
        self.work_time = timedelta(hours=8.0)
        self.lunch_time = timedelta(hours=1.0)
        self.min_time_bf_dinner = timedelta(hours=4.0) # Обед минимум через 4 часа работы
        self.type = "A"
        self.state = 0
        self.schedule = []
        self.hunger = True
        Driver_A.count += 1
        self.id = Driver_A.count
        Driver_A.all_drivers.append(self)

        # нельзя обедать во ремя час пика и незавершенного маршрута

    def __repr__(self):
        return str(self.id) + " " + str(self.type) + " " + str(self.bus)

    def start(self, time, bus, all_buses_exit_schedule):
        self.work_start_time = time
        self.start_time = time
        self.state = 1
        self.bus = bus
        self.bus.driver = self
        
        self.schedule.append([time, "Выход"])
        self.bus.schedule.append([time, "Выход с автопарка"])
        all_buses_exit_schedule.append([self, time])

     
    
    def check(self, current_time, free_buses, busy_buses, all_buses_exit_schedule):
        if self.state != 0:
           
            if (current_time - self.work_start_time) >= (self.work_time+self.lunch_time):
                self.state = 0
                self.schedule.append([current_time, "Уход"])
                self.bus.driver = None
                free_buses.append(self.bus)
                busy_buses.remove(self.bus)
                self.bus.schedule.append([current_time, "Возвращение в автопарк"])
                
            if self.state == 1:
                if current_time - self.start_time >= route_time*2:
                    self.bus.schedule.append([current_time, "Возвращение в автопарк"])
                    if self.need_rest(current_time):
                        self.state = 2
                        self.start_time = current_time
                        self.schedule.append([current_time, "Перерыв"])
                      
                    else:
                        self.bus.schedule.append([current_time, "Выход с автопарка"])
                       
                        all_buses_exit_schedule.append([self, current_time])
                        self.start_time = current_time
            elif self.state == 2:
                if current_time - self.start_time >= self.lunch_time:
                    self.hunger = False
                    self.state = 1
                    self.start_time = current_time
                    self.schedule.append([current_time, "Выход"])
                    self.bus.schedule.append([current_time, "Выход с автопарка"])
                    all_buses_exit_schedule.append([self, current_time])
                    
            

    def need_rest(self, current_time):
        if self.work_start_time:
            if (self.hunger == True) and (current_time - self.work_start_time >= self.min_time_bf_dinner):
                return True
        return False  

class Driver_B:
    count = 0
    all_drivers = []

    def __init__(self, bus=None):
        self.work_days = []
        self.bus = bus
        self.work_time = timedelta(hours=12.0)
        self.lunch_time = timedelta(minutes=15.0)
        self.lunch_time_interval = timedelta(hours=2.0) # минимум
        self.type = "B"
        self.state = 0
        self.schedule = []
        self.hunger = True
        Driver_B.count += 1
        self.id = Driver_B.count
        Driver_B.all_drivers.append(self)
        # Обед во время одного из перерывов
        # 1 сутки работает, 2 отдыхает
    
    def __repr__(self):
        return str(self.id) + " " + str(self.type) + " " + str(self.bus)

    def start(self, time, bus, all_buses_exit_schedule):
        self.work_start_time = time
        self.start_time = time
        self.state = 1
        self.bus = bus
        self.bus.driver = self
        
        
        self.schedule.append([time, "Выход"])
        self.bus.schedule.append([time, "Старт с автопарка"])
        all_buses_exit_schedule.append([self, time])



    def check(self, current_time, free_buses, busy_buses, all_buses_exit_schedule):
        if self.state != 0:
            if (current_time - self.work_start_time) >= (self.work_time+5*self.lunch_time):
                self.state = 0
                self.schedule.append([current_time, "Уход"])
                self.bus.driver = None
                free_buses.append(self.bus)
                busy_buses.remove(self.bus)
                self.bus.schedule.append([current_time, "Возвращение в автопарк"])
                
            if self.state == 1:
                if current_time - self.start_time >= route_time*2:
                    self.bus.schedule.append([current_time, "Возвращение в автопарк"])
                    if self.need_rest(current_time):
                        self.state = 2
                        self.start_time = current_time
                        self.schedule.append([current_time, "Перерыв"])
                        
                    else:
                        self.bus.schedule.append([current_time, "Выход с автопарка"])
                        all_buses_exit_schedule.append([self, current_time])
                        self.start_time = current_time
                       
            elif self.state == 2:
                if current_time - self.start_time >= self.lunch_time:
                    self.hunger = False
                    self.state = 1
                    self.start_time = current_time
                    self.bus.schedule.append([current_time, "Выход с автопарка"])
                    all_buses_exit_schedule.append([self, current_time])
                   

    def need_rest(self, current_time):
        if self.start_time:
            if (self.hunger == True) and (current_time - self.start_time >= self.lunch_time_interval):
                return True
        return False  

class Bus:
    count = 0
    all_buses = []

    def __init__(self):
        Bus.count += 1
        self.schedule = []
        self.id = Bus.count
        self.driver = None
        Bus.all_buses.append(self)
        
    def __repr__(self):
        return str(self.id)

def get_drivers_choice(drivers_count=5):
    drivers = []
    day_index_list_A = [1, 2, 3]
    day_index_list_B = [1, 2, 3]
    day_index_A = 0
    day_index_B = 0
    drivers_B = []
    for drivers_B_Count in range(0, drivers_count+1): # выбирается количество водителей типа B
        
        if drivers_B_Count != 0:
            
            driver_B = Driver_B()
            driver_B.work_days = [day_index_list_B[day_index_B]+i for i in range(0, 4, 3)]
            if day_index_B == 2:
                day_index_B = 0
            else:
                day_index_B += 1  
            drivers_B.append(driver_B)
         
        drivers_A = []
        day_index_A = 0
        for i in range(0, drivers_count-drivers_B_Count): # выбирается количество водителей типа A
          
        
            driver_A = Driver_A()
           
            driver_A.work_days = [day_index_list_A[day_index_A]+j for j in range(5)]
            
            if day_index_A == 2:
                day_index_A = 0
            else:
                day_index_A += 1
            drivers_A.append(driver_A)
       
        append_list_A = drivers_A.copy()
        append_list_B = drivers_B.copy()
        drivers.append([append_list_A, append_list_B])
    return drivers

def get_driver_by_type(drivers_list: list, type: str):
    if len(drivers_list) == 0:
        return False
    for driver in drivers_list:
        if driver.type == type:
            drivers_list.remove(driver)
            return driver

# переменные
buses_count = 10
drivers_count = 10
route_time = timedelta(hours=1.0)

# часы работы автопарка
start_time = timedelta(hours=6.0)
finish_time = timedelta(days=1, hours=3.0)

# часы пик ПН-ПТ
peak_hour_starts = [timedelta(hours=7.0), timedelta(hours=17.0)]
peak_hour_finishes = [timedelta(hours=9.0), timedelta(hours=19.0)]

buses_schedule = []
A_drivers = []
B_drivers = []
peak_interval = timedelta(hours=1.0)

population = []


def get_schedule(buses_count=10, drivers_count=10, route_time=timedelta(hours=1.0)):
    g = 0

    while g < 10:
        day_index = 1

        all_buses_exit_schedule = []
        free_drivers = []
        free_buses = []
        busy_buses = []
        planned_time = []
        drivers_A_count = 0
        drivers_B_count = 0
        current_time = start_time

        drivers = get_drivers_choice(drivers_count)
        
        # сбор водителей этого дня в массив и подсчет количества водителей разных типов
  
        for drivers_list in drivers[g]:
            for driver in drivers_list:
                if day_index in driver.work_days:
                    free_drivers.append(driver)
                    if driver.type == "A":
                        drivers_A_count += 1
                    if driver.type == "B":
                        drivers_B_count += 1

        # сбор автобусов в массив
        for i in range(buses_count):
            free_buses.append(Bus())

        # счетчик времени прибавляет по 5 мин. пока не дойдет до закрытия автопарка
        while current_time <= finish_time:
         
            
            # в начале проверяем, если много водителей, разделяем их равномерно по часам пик
            # водители типа А
            if current_time == start_time and len(free_buses) != 0:
                if drivers_A_count > 3:
                    offset = timedelta(minutes=5.0)
                    if drivers_A_count == 4:
                        planned_time.append([start_time+offset, "A"])
                    for i in range((drivers_A_count-3)//2):
                        planned_time.append([start_time+offset, "A"])
                        offset += timedelta(minutes=5.0)
                    offset = timedelta(minutes=5.0)
                    for i in range(len(free_drivers)-((drivers_A_count-3)//2)):
                        planned_time.append([peak_hour_starts[1]+offset, "A"])
                        offset += timedelta(minutes=5.0)
            # водители типа B
                if drivers_B_count > 1:
                    offset = timedelta(minutes=5)
                    for i in range((drivers_B_count-1)):
                        planned_time.append([start_time+timedelta(hours=9.0)+offset, "B"])
                        offset += timedelta(minutes=10.0)
            
            if len(planned_time) != 0:
                for p_time in planned_time:
                    
                    if current_time == p_time[0] and len(free_buses) != 0 and len(free_drivers) !=0:
                        driver = get_driver_by_type(free_drivers, p_time[1])
                        if driver:
                            bus =  free_buses[0]
                            driver.start(current_time, bus, all_buses_exit_schedule)
                            busy_buses.append(bus)
                            del free_buses[0]

            if ((current_time == start_time) or (current_time == start_time+timedelta(hours=9.0)) or\
                (current_time == start_time+timedelta(hours=11.0))) and len(free_buses) != 0:
                
                get_drivers_list = [get_driver_by_type(free_drivers, "A"), get_driver_by_type(free_drivers, "B")]
                if current_time != start_time:
                    get_drivers_list = get_drivers_list[:1]
                    
                for driver in get_drivers_list:
                    if driver:
                        bus =  free_buses[0]
                        driver.start(current_time, bus, all_buses_exit_schedule)
                        busy_buses.append(bus)
                        del free_buses[0]

            # проверка состояния активных автобусов 
            if len(busy_buses) != 0:
                for bus in busy_buses:
                    bus.driver.check(current_time, free_buses, busy_buses, all_buses_exit_schedule)
            
            # шаг по времени на 5 мин.
            current_time += timedelta(minutes=5.0)

        summ = timedelta(hours=0.0)
        day_interval = timedelta(hours=0.0)
        for i in range(1, len(all_buses_exit_schedule)):
            summ += (all_buses_exit_schedule[i][1] - all_buses_exit_schedule[i-1][1])
        day_interval = summ/len(all_buses_exit_schedule)
      


        peak_summ = timedelta(hours=0.0)
        for i in range(1, len(all_buses_exit_schedule)):
            if timedelta(hours=7.0) < all_buses_exit_schedule[i][1] < timedelta(hours=9.0) or \
                timedelta(hours=17.0) < all_buses_exit_schedule[i][1] < timedelta(hours=19.0):
                peak_summ += (all_buses_exit_schedule[i][1] - all_buses_exit_schedule[i-1][1])
        peak_interval = peak_summ/len(all_buses_exit_schedule)

        buses_schedule = all_buses_exit_schedule
        population.append(buses_schedule)
        
        for driver in Driver_A.all_drivers:
            A_drivers.append(driver)
        for driver in Driver_B.all_drivers:
            B_drivers.append(driver)
        
        if peak_interval > timedelta(minutes=5.0):
            drivers_count += 2
            buses_count += 1

        g += 1


    GENES_LENGTH = 1000    
    MAX_GENERATIONS = 50    # максимальное количество поколений

    class Individual():
        def __init__(self, chromosome):
            self.chromosome = chromosome

    class Population():
        def __init__(self, individuals):
            self.individuals = individuals

        def get_population(self):
            return [ind.chromosome for ind in self.individuals]
        
        def get_fit(self, individual):
            fit = 0
            for i in range(len(individual.chromosome)):
                try:
                    chrome = individual.chromosome[i][1]
                except:
                    try:
                        ind = individual.chromosome[i]
                    except:
                        ind = 0
                    if ind != 0:
                        del individual.chromosome[i]
                    chrome = 0

                if chrome != 0:
                    if (timedelta(hours=7.0) < chrome < timedelta(hours=9.0)) or \
                        ((timedelta(hours=17.0) < chrome < timedelta(hours=19.0))):
                        fit += 20
                    if (timedelta(hours=15.00) < chrome < timedelta(hours=16.00)):
                        fit += 12
                else:
                    fit -= 1
            return fit
                    
        def get_best(self):
            population = self.individuals
            best_id = 0
            best_fit = 0
            for i in range(len(self.individuals)):
                if self.get_fit(self.individuals[i]) >= best_fit:
                    best_fit, best_id = self.get_fit(population[i]), i
            return [best_id, best_fit]

    def selection(population): # отбор
        best_id = population.get_best()[0]
        population = population.get_population()
        offspring = [population[best_id]] # потомки
        chosen_ids = [best_id] # id отобранных потомков
        del population[best_id]
        
        rand_id = random.randint(0, len(population)-1)
        offspring.append(population[rand_id])
        chosen_ids.append(rand_id)
        del population[rand_id] 
        
        offspring.append(chosen_ids)
        return offspring

    def crossover(offspring): # кроссинговер
        s = random.randint(2, len(offspring[0])-3) # учитываем границы 
        offspring[0][s:], offspring[1][s:] = offspring[1][s:], offspring[0][s:] 
        return [offspring[0], offspring[1]]

    def mutation(mutant): # gen_p_mut вероятность мутации отдельного гена
        rand_id = random.randint(0, len(mutant)-1)
        mutant[rand_id] = 0 if mutant[rand_id] == 1 else 1
        return mutant


    individuals = []
    for el in population:
        ind = Individual(el)
        individuals.append(ind)

    new_population = Population(individuals)
    population_count = 0
    maxFitnessValues = []
    meanFitnessValues = []

    best_index = 0
    top_population = []

    while new_population.get_best()[1] < GENES_LENGTH and population_count < MAX_GENERATIONS:
        # отбор
        result = selection(new_population)
        sel, ids = [result[0], result[1]], result[2] 

        # скрещивание
        cross = crossover(sel)
        for id in range (len(new_population.get_population())):
            if id not in ids:
                cross.append(new_population.get_population()[id])
        
        # мутация
        rand_ind = random.randint(0, len(cross)-1)
        cross[rand_ind] = mutation(cross[rand_ind])

        individuals = []
        for el in cross:
            ind = Individual(el)
            individuals.append(ind)
        new_population = Population(individuals)
        top_population = new_population.get_population()

        population_count += 1
        fitnessValues = [new_population.get_fit(ind) for ind in new_population.individuals]
    

        maxFitness = max(fitnessValues)
        meanFitness = sum(fitnessValues) / len(new_population.get_population())
        maxFitnessValues.append(maxFitness)
        meanFitnessValues.append(meanFitness)
     
        best_index = fitnessValues.index(max(fitnessValues))



    peak_summ = timedelta(hours=0.0)
    for i in range(1, len(top_population)):
        if timedelta(hours=7.0) < top_population[i][0][1] < timedelta(hours=9.0) or \
            timedelta(hours=17.0) < top_population[i][0][1] < timedelta(hours=19.0):
            peak_summ += (top_population[i][0][1] - top_population[i-1][0][1])
    peak_interval = peak_summ/len(top_population)



    return [top_population, peak_interval]


app = QApplication(sys.argv)
input_window = DataInputWindow()
input_window.show()
sys.exit(app.exec_())

SystemExit: 0

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
